In [9]:
import pandas as pd
import re
import requests
import certifi
from bs4 import BeautifulSoup

In [10]:
url='https://open.bymadata.com.ar/#/technical-detail-bond?symbol=AL35D'
response=requests.get(url,verify=False)
soup=BeautifulSoup(response.text, 'html.parser')
soup

c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'open.bymadata.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<title>BYMADATA - Cotizaciones en Tiempo Real | BYMA Bolsas y Mercados Argentinos</title>
<meta content="BYMA te acerca la informaciÃ³n mÃ¡s relevante del Mercado de Capitales argentino en un solo lugar." name="description"/>
<meta content="x9zbmyu0rbwsxvklczmdu6ujdbq5s8" name="facebook-domain-verification"/>
<!-- base url -->
<base href="/"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"/>
<!-- #CSS Links -->
<!-- Basic Styles -->
<link href="assets/css/bootstrap.min.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="assets/css/font-awesome.min.css" media="screen" rel="stylesheet" type="text/css"/>
<!-- SmartAdmin Styles : Caution! DO NOT change the order -->
<link href="assets/css/smartadmin-production-plugins.min.css" media="screen" rel="stylesheet" type="text/css"/>